Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium pandas google


In [61]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback
import re

In [66]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

p_elementList= []

# Cuntion for Getting Date from Content
def extractDate(string):
    try:
       # Define a regular expression for the date format DD/MM/YYYY and optional time
        date_time_pattern = re.compile(
            r"^\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b"  # Matches dates like DD/MM/YYYY at the start of the string
            r"(?:\s+\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM)?\s*(?:ET|WT)?\b)?$",  # Optionally matches times with optional AM/PM and time zone
            re.IGNORECASE  # Make the pattern case-insensitive
        )
        # Check if the entire string matches the pattern
        match = date_time_pattern.match(string)
        return bool(match)
    except:
        print("Error accessing a text:", e)
    return False

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        H1_Title = ''
        H2_Title = ''
        H3_Title = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        H2_elements = driver.find_elements(By.TAG_NAME, "h2")
        H3_elements = driver.find_elements(By.TAG_NAME, "h3")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
            
        # Getting Title
        for h1 in H1_elements:
            try:
                H1_Title = h1.text
                if (H1_Title == ''):
                    for h2 in H2_elements:
                        H2_Title = h2.text
                        if (H2_Title == ''):
                            for h3 in H3_elements:
                                H3_Title = h3.text
                                break
                        break
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.startswith('By '):
                    authors = value.replace('By ', '') 
                    words = authors.split()
                    if len(words) <= AUTHOR_MAX_WORDS and not authors:
                        authors = value
                    else:
                        authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                if (extractDate(value)):
                    date = value
                    continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'H1Title' : H1_Title,'H2Title' : H2_Title,'H3Title' : H3_Title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright } 

def getPElementList(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        readMoreUrl = ''
        date = ''
        driver.get(url)
        p_elements = driver.find_elements(By.TAG_NAME, "p")

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                p_elementList.append({'p_element': value,'title' : title,  'source_domain': source_domain, 'url' : url, })

            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return 

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')


In [67]:
# define query String
#queryString = 'Did Joe Biden tell voters to stay home and not vote?'
queryString = 'Is Michael Saylor saying he will give away free bitcoin?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [68]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        data = getContent(driver,url)
        print(data)
        if (data['H1Title'] !='' or data['H2Title'] !='' or data['H3Title'] !=''):
            print('pass')
        #dataSet.append(data)
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)


Scripting from => https://news.bitcoin.com/michael-saylor-warns-of-deepfake-bitcoin-giveaway-scams-featuring-him-and-microstrategy/
{'authors': '', 'date': '', 'H1Title': '', 'H2Title': '', 'H3Title': '', 'content': '\u200c\n\n\u200c \u200c\n\n\u200c\n\n\u200c\n\n\u200c \u200c\n\n\u200c \u200c\n\n\u200c\n\n\u200c\n\n\u200c Get the newsletter directly to your inbox \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c ', 'source_domain': 'news.bitcoin.com', 'url': 'https://news.bitcoin.com/michael-saylor-warns-of-deepfake-bitcoin-giveaway-scams-featuring-him-and-microstrategy/', 'copyright': ''}
pass
Scripting from => https://www.reddit.com/r/CryptoCurrency/comments/18os1kh/why_is_a_blatant_scam_depicting_michael_saylor/
{'authors': '', 'date': '', 'H1Title': 'Why is a blatant scam depicting Michael Saylor asking for Bitcoin allowed to be shown as an advertisement on Youtube?', 'H2Title': '', 'H3Title': '', 'content': ' Youtube is running advertisements that are blatant scams. I\'ve r

KeyboardInterrupt: 

In [47]:
df

,authors,date,title,content,source_domain,url,copyright
0,,,,,news.bitcoin.com,https://news.bitcoin.com/michael-saylor-warns-...,
1,,,Why is a blatant scam depicting Michael Saylor...,Youtube is running advertisements that are bl...,www.reddit.com,https://www.reddit.com/r/CryptoCurrency/commen...,
2,,,Michael Saylor scam ad,Just saw a Saylor ad offering to give out fre...,www.reddit.com,https://www.reddit.com/r/Bitcoin/comments/18r6...,
3,,,Michael Saylor Bitcoin Scam.,I lost 2 Bitcoin! Don’t fall for it! They pla...,www.reddit.com,https://www.reddit.com/r/CryptoScams/comments/...,
4,,,"Fake Michael Saylor audio leak going around, h...",So a fake supposedly leak audio is going arou...,www.reddit.com,https://www.reddit.com/r/CryptoCurrency/commen...,
...,...,...,...,...,...,...,...
178,,,,,www.youtube.com,https://www.youtube.com/watch?v=kuUqUI57Kyc,
179,,,,,www.youtube.com,https://www.youtube.com/watch?v=cjg79mCPEI8,
180,,,,,www.youtube.com,https://www.youtube.com/watch?v=Q8IIuKXBsXs,
181,,,,,www.youtube.com,https://www.youtube.com/watch?v=HlGnaxqCuxg,


In [57]:
save_to_csv_with_pandas(df, queryString.replace('?','.')+ 'csv')

In [60]:
filtered_df = df[df['date'].notna() & (df['date'] != '')]
filtered_df['date']

39                  20:13 • Jun 29, 2023 • Altcoin Daily
46                           01:16:31 | Robert Breedlove
62     MicroStrategy repaid its $205M Silvergate loan...
74     Lex Fridman [timestamp 3:45:49]: Well the inte...
98                      December 19, 2023 — 01:23 am EST
102                                   15 Feb 2024, 16:55
125               CRYPTO ECOSYSTEMS 01/11/24 12:22PM EST
132    The Bitcoin price dropped 1.3% in the last 24 ...
Name: date, dtype: object